## Configuring filter pipelines

This Jupyter notebook explains the workflow of setting up and configuring a ground point filtering pipeline. This is an advanced workflow for users that want to define their own filtering workflows. For basic use, preconfigured pipelines are (or rather: will be) provided by `adaptivefiltering`. As always, we first need to import our library:

In [1]:
import adaptivefiltering

Also, we need to load at least one data set which we will use to interactively preview our filter settings. Note that for a good interactive experience with no downtimes, you should restrict your datasets to a reasonable size (see the working with datasets demo notebook for how to do it).

In [4]:
#dataset = adaptivefiltering.DataSet(filename="data/500k_NZ20_Westport.laz")
dataset = adaptivefiltering.DataSet(filename="data/uls_thingstaette.las")


The main pipeline configuration is done by calling the `pipeline_tuning` function with your dataset as the parameter. This will open the interactive user interface which waits for your user input until you hit the *Finalize* button. The configured filter is then accessible as the Python object `pipeline`:

In [5]:
pipeline = adaptivefiltering.pipeline_tuning(dataset)

/home/gwydion/anaconda3/envs/adaptivefiltering/lib/python3.9/site-packages/adaptivefiltering/data/uls_thingstaette.las
/home/gwydion/anaconda3/envs/adaptivefiltering/lib/python3.9/site-packages/adaptivefiltering/data/uls_thingstaette.las


AppLayout(children=(VBox(children=(HTML(value='Interactive pipeline configuration:', layout=Layout(grid_area='…

This pipeline can e.g. be written to disk with the following command. We typically use the extension `json` for filters. It stands for *JavaScript Object Notation* and is a widely used format for storing custom data structures:

In [6]:
adaptivefiltering.save_filter(pipeline, "myfilter.json")

In the above, we covered only the basic use of the `pipeline_tuning` function. For other use cases, additional parameters can be passed. If you want to edit an existing ground point filtering pipeline, you can load it using the `load_filter` function and then pass it to `pipeline_tuning`. The pipeline object returned by `pipeline_tuning` will be a new object - no implicit changes of the loaded pipeline object will occur:

In [ ]:
old_pipeline = adaptivefiltering.load_filter("myfilter.json")
edited_pipeline = adaptivefiltering.pipeline_tuning(dataset, pipeline=old_pipeline)

If you want to inspect multiple data sets in parallel while tuning a pipeline, you can do so by passing a list of datasets to the `pipeline_tuning` function. Note that `adaptivefiltering` does currently not parallelize the execution of filter pipeline execution which may have a negative impact on wait times while tuning with multiple parameters.

In [ ]:
pipeline2 = adaptivefiltering.pipeline_tuning(datasets=[dataset, dataset])